In [17]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

In [22]:
dataset = load_dataset("nbertagnolli/counsel-chat")

Repo card metadata block was not found. Setting CardData to empty.


In [28]:
print(dataset.keys())

dict_keys(['train'])


In [29]:
train_data = dataset["train"]

In [30]:
def formatting_func(example):
    return f"### {example['dialogue_id']}: {example['utterance']}"

In [31]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf", padding_side="left", add_eos_token=True, add_bos_token=True)


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Llama-2-7b-hf.
401 Client Error. (Request ID: Root=1-660a29e8-06ee30d35e34f1db6c0bfcb6;34d71320-128c-4a53-9b8a-b74c614995ad)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-2-7b-hf/resolve/main/config.json.
Repo model meta-llama/Llama-2-7b-hf is gated. You must be authenticated to access it.

In [ ]:
def generate_and_tokenize_prompt(prompt):
    return tokenizer(formatting_func(prompt))

In [ ]:
tokenized_train_dataset = train_data.map(generate_and_tokenize_prompt)

In [ ]:
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-hf", quantization_config={"load_in_4bit": True})


In [ ]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
config = LoraConfig(
    r=32,
    lora_alpha=64,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    bias="none",
    lora_dropout=0.05,
    task_type="CAUSAL_LM",
)


In [ ]:
model = get_peft_model(model, config)

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

for epoch in range(3):
    model.train()
    for batch in tokenized_train_dataset:
        optimizer.zero_grad()
        inputs = {k: v.to(torch.device("cuda")) for k, v in batch.items()}
        outputs = model(**inputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()


In [ ]:
model.save_pretrained("fine_tuned_llama")